## 多任务模型: 模糊图片 → SR修复 → 分类

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import os

from NetSet import SRNet, ClassifyNet
from datasets import CIFAR10Dataset, Fuzz

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_dir = "results/multi"
os.makedirs(save_dir, exist_ok=True)

cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std  = (0.2023, 0.1994, 0.2010)
normalize = transforms.Normalize(cifar10_mean, cifar10_std)

class MultiTaskNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.sr = SRNet()
        self.classifier = ClassifyNet()
        self.normalize = normalize

    def forward(self, x):
        sr_out = self.sr(x)
        sr_norm = self.normalize(sr_out)
        cls_out = self.classifier(sr_norm)
        return sr_out, cls_out

## 加载预训练权重

In [2]:
model = MultiTaskNet().to(device)

model.sr.load_state_dict(
    torch.load("results/task1/model_epoch_100.pth", map_location=device))

model.classifier.load_state_dict(
    torch.load("results/task2/model_epoch_100.pth", map_location=device))

print("预训练权重加载完成")

预训练权重加载完成


## 联合微调训练

In [ ]:
batch_size = 64
epochs = 50 
lr = 0.0005
momentum = 0.9
lambda_sr = 1.0
lambda_cls = 1.0

train_dataset = CIFAR10Dataset(root_dir='./DS/CIFAR10', train=True)
test_dataset  = CIFAR10Dataset(root_dir='./DS/CIFAR10', train=False)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

fuzz = Fuzz(scale_factor=2)

inv_mean = torch.tensor(cifar10_mean).view(3,1,1).to(device)
inv_std  = torch.tensor(cifar10_std).view(3,1,1).to(device)

def denormalize(x):
    return x * inv_std + inv_mean

sr_criterion  = nn.L1Loss()
cls_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(epochs):
    model.train()
    epoch_sr_loss = 0
    epoch_cls_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        clean = denormalize(inputs)
        blurry = torch.stack([fuzz(img) for img in clean]).to(device)

        optimizer.zero_grad()
        sr_out, cls_out = model(blurry)

        loss_sr  = sr_criterion(sr_out, clean)
        loss_cls = cls_criterion(cls_out, labels)
        loss = lambda_sr * loss_sr + lambda_cls * loss_cls
        loss.backward()
        optimizer.step()

        epoch_sr_loss  += loss_sr.item()
        epoch_cls_loss += loss_cls.item()
        _, predicted = torch.max(cls_out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_sr  = epoch_sr_loss / len(train_loader)
    avg_cls = epoch_cls_loss / len(train_loader)
    train_acc = 100 * correct / total

    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            clean = denormalize(inputs)
            blurry = torch.stack([fuzz(img) for img in clean]).to(device)
            _, cls_out = model(blurry)
            _, predicted = torch.max(cls_out.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_acc = 100 * test_correct / test_total

    print(f"Epoch [{epoch+1:3d}/{epochs}] SR: {avg_sr:.5f} | CLS: {avg_cls:.5f} | Train: {train_acc:.2f}% | Test: {test_acc:.2f}%")

torch.save(model.state_dict(), f"{save_dir}/multi_model.pth")
print(f"保存至 {save_dir}/")

Epoch [  1/50] SR: 0.04343 | CLS: 0.71679 | Train: 76.06% | Test: 76.60%
Epoch [  2/50] SR: 0.04209 | CLS: 0.64683 | Train: 78.06% | Test: 78.00%
Epoch [  3/50] SR: 0.04169 | CLS: 0.60678 | Train: 79.62% | Test: 78.49%
Epoch [  4/50] SR: 0.04109 | CLS: 0.59395 | Train: 79.85% | Test: 79.12%
Epoch [  5/50] SR: 0.04091 | CLS: 0.57188 | Train: 80.70% | Test: 78.10%
Epoch [  6/50] SR: 0.04052 | CLS: 0.55962 | Train: 80.99% | Test: 79.30%
Epoch [  7/50] SR: 0.04081 | CLS: 0.54611 | Train: 81.54% | Test: 78.84%
Epoch [  8/50] SR: 0.04044 | CLS: 0.52826 | Train: 81.96% | Test: 79.02%
Epoch [  9/50] SR: 0.04043 | CLS: 0.52224 | Train: 82.19% | Test: 78.94%
Epoch [ 10/50] SR: 0.04038 | CLS: 0.51672 | Train: 82.30% | Test: 79.40%
Epoch [ 11/50] SR: 0.03972 | CLS: 0.50392 | Train: 82.83% | Test: 79.67%
Epoch [ 12/50] SR: 0.03971 | CLS: 0.49567 | Train: 83.25% | Test: 80.33%
Epoch [ 13/50] SR: 0.04057 | CLS: 0.48474 | Train: 83.59% | Test: 80.12%
Epoch [ 14/50] SR: 0.03991 | CLS: 0.48316 | Train: 

## 对比: 模糊直接分类 vs 多任务(SR+分类)

In [1]:
baseline = ClassifyNet().to(device)
baseline.load_state_dict(torch.load("results/task2/model_epoch_100.pth", map_location=device))
baseline.eval()
model.eval()

acc_baseline = 0
acc_multi = 0
acc_clean = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        clean = denormalize(inputs)
        blurry = torch.stack([fuzz(img) for img in clean]).to(device)

        upscaled = nn.functional.interpolate(blurry, scale_factor=2, mode='bicubic', align_corners=False)
        upscaled_norm = normalize(upscaled)
        pred_baseline = baseline(upscaled_norm).argmax(dim=1)

        _, cls_out = model(blurry)
        pred_multi = cls_out.argmax(dim=1)

        pred_clean = baseline(inputs).argmax(dim=1)

        total += labels.size(0)
        acc_baseline += (pred_baseline == labels).sum().item()
        acc_multi    += (pred_multi == labels).sum().item()
        acc_clean    += (pred_clean == labels).sum().item()

acc_baseline = 100 * acc_baseline / total
acc_multi    = 100 * acc_multi / total
acc_clean    = 100 * acc_clean / total

print(f"清晰图直接分类:       {acc_clean:.2f}%")
print(f"低分辨率图直接分类:  {acc_baseline:.2f}%")
print(f"低分辨率图 → 超分 → 分类:       {acc_multi:.2f}%")
print(f"提升: {acc_multi - acc_baseline:+.2f}%")

NameError: name 'ClassifyNet' is not defined